In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp

In [13]:
def make_function(equation_str):
    x,y = sp.symbols('x y')

    expr = sp.simplify(equation_str)

    return expr

def euler_method(function, step_size, y_0, x_0, x_end):
    result = function.subs({"x": x_0, "y": y_0})

    x_values = np.arange(x_0, x_end+step_size, step_size)
    y_values=[y_0]

    print(x_values)

    for i in range(len(x_values)-1):
        y_value = y_values[i] + (step_size*function.subs({"x": x_values[i], "y": y_values[i]}))
        y_values.append(y_value)
    
    plot_euler(function, x_values,y_values)

def exact_solve(equation_str, x_0, x_end, y_0, step_size):

    x,y = sp.symbols('x y')

    # Define the symbols
    x = sp.Symbol('x')
    y = sp.Function('y')

    eq = sp.Eq(y(x).diff(x), sp.sympify(equation_str))

    # Solve the ODE
    solution = sp.dsolve(eq, y(x))

    C = sp.Symbol('C')
    constant_value = sp.solve(solution.rhs.subs(x, x_0) - y_0, C)
    print(constant_value)

    # Particular solution
    particular_solution = solution.subs(C, constant_value)

    return particular_solution

# Function to plot Euler's method results
def plot_euler(function, x_values, y_values):

    plt.figure(figsize=(8, 5))
    plt.plot(x_values, y_values, marker='o', linestyle='-', label="Euler's Method Approximation")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title(f"Euler's Method for dy/dx = {str(function)}")
    plt.legend()
    plt.grid(True)
    plt.show()

exact_solve("x+y", x_0=0, x_end=0.5, y_0=1,step_size = 0.1)

function=make_function("x+y")

euler_method(function=function, step_size = 0.1,y_0=1,x_0=0,x_end=0.5)


[]


SympifyError: SympifyError: []

In [ ]:
# david's example



IndexError: list index out of range